# Managing Physical Units

- In GDM, all physical quantities are wrapped in pint objects (https://pypi.org/project/Pint/).
- Users import physical quantities when building new models or updating model parameters.
  - Units are validated for all physical quantities.
  - All quantities are imported from `gdm.quantities`.

## Creating Quantities


In [1]:
from gdm.quantities import ReactivePower

power = ReactivePower(800, "kilovar")
print(power)

800 kilovar


The `to` method can be used for unit conversion

In [2]:
print(power.to("var"))

800000.0 var


The quantitiy models can be used with list and numpy arrays

In [3]:
powers = ReactivePower([400, 200, 100], "kilovar")
print(powers)
print(powers.to("var"))

[400 200 100] kilovar
[400000.0 200000.0 100000.0] var


In certain cases, additional validation may be performed. e.g. PV systems can only provide active power hence represented by PossitiveActivePower

In [4]:
from gdm.quantities import PositiveActivePower

try:
    PositiveActivePower(-700, "kilowatt")
except Exception as e:
    print(e)

Active power (-700, kilowatt) must be positive.


## Validation Limitations

```{warning}
Unit validation does not occur until the quantity is added to a data model. In the example below, the `rated_voltage` object is built with incorrect units. It is only validated when we instantiate the `DistributionBus` component.
```


In [5]:
from gdm.distribution.enums import LimitType, Phase, VoltageTypes
from gdm.distribution.components import DistributionSubstation
from gdm.distribution.common.limitset import VoltageLimitSet
from gdm.distribution.components import DistributionFeeder
from gdm.distribution.components import DistributionBus
from gdm.quantities import PositiveVoltage
from infrasys import Location

rated_voltage = PositiveVoltage(400, "ampere")

/opt/homebrew/Caskroom/miniconda/base/envs/gdm2/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [6]:
bus = DistributionBus(
    voltage_type=VoltageTypes.LINE_TO_LINE,
    phases=[Phase.A, Phase.B, Phase.C],
    rated_voltage=rated_voltage,
    name="DistBus1",
    substation=DistributionSubstation.example(),
    feeder=DistributionFeeder.example(),
    voltagelimits=[
        VoltageLimitSet(
            limit_type=LimitType.MIN, value=PositiveVoltage(400 * 0.9, "volt")
        ),
        VoltageLimitSet(
            limit_type=LimitType.MAX, value=PositiveVoltage(400 * 1.1, "volt")
        ),
    ],
    coordinate=Location(x=20.0, y=30.0),
)

ValidationError: 1 validation error for DistributionBus
rated_voltage
  Assertion failed, Unit must be compatible with volt [type=assertion_error, input_value=<Quantity(400, 'ampere')>, input_type=PositiveVoltage]
    For further information visit https://errors.pydantic.dev/2.10/v/assertion_error